In [1]:
import numpy as np
import math as mt
import pandas as pd
import math
import random
import qiskit as q
from qiskit import IBMQ
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from qiskit.quantum_info import Statevector
from qiskit.visualization import *
import qiskit.quantum_info as qi
from qiskit.circuit import ParameterVector
from qiskit.compiler import transpile, assemble
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit import quantum_info
from  qiskit.circuit.library import CU3Gate
from qiskit.circuit.library import UGate
#from ibm_quantum_widgets import *


#from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.circuit.library import EfficientSU2
#from qiskit.aqua.operators import WeightedPauliOperator, MatrixOperator
#from qiskit.aqua.components.optimizers import SLSQP
from qiskit import Aer , transpile, assemble

In [2]:
q.__version__

'0.20.2'

In [3]:
from qiskit import IBMQ
IBMQ.save_account('d0d8dddef75223e99f5ce3294800b067e878ea80d5616de23a26b741f2aee0a9fb39fdfdf8e7c7f77c42f931c59a12e3ae1d9e0b2bc9d65b256f6bf6f54d6cda',overwrite=True)


## Function to Build Quantum Circuit

In [4]:
#The circuit will be initilised wioth zeros and no superposition will be applied hence no H gates
def BuildCircuit(locs):
    print ("----------Staring the Process and Allocating Qubits -------------")
    print ("                                                                 ")
    Total_Qubits = locs*(locs - 1)
    print ("Total Qubits allocated is :   " , Total_Qubits)
    qc = q.QuantumCircuit(Total_Qubits)
    qc.draw(output="mpl")
    #for idx in range(Total_Qubits):
    #    qc.h(idx)
    #qc.draw(output="mpl")
    return qc

## Define Operators

In [5]:
from qiskit.opflow import I, X, Y, Z
from qiskit.quantum_info.operators import Operator, Pauli
#I=np.array([[1,0],[0,1]])
#Z=np.array([[1,0],[0,-1]])

#II=np.kron(I,I)
#IZ=np.kron(I,Z)
#ZI=np.kron(Z,I)
#ZZ=np.kron(Z,Z)


In [6]:
#Function for Defining Coefficient ZZ_{ij}
def coff_zz(a,b,qubits):
    n=qubits
    #a=a-1
    j=b
    for i in reversed(range(n)):
        mar_arr = ['I']*n
        #print(mar_arr)
        mar_arr[a] = 'Z'
        #print(mar_arr)
        mar_arr[j]='Z'
        #print(mar_arr)
        s = ""
        b = ""
        for k in range(n -1):
             s+= mar_arr[k]+'^'
        #print(s)
        for l in range(n -1):
            b+=')'
        final_eq = s+mar_arr[n-1]
        print(final_eq)
        break
    return eval(final_eq)

In [7]:
#Function for Defining Coefficient Z_{i}
def coff_z(a,qubits):
    n=qubits
    #a=a-1
   
    for i in reversed(range(n)):
        mar_arr = ['I']*n
        #print(mar_arr)
        mar_arr[a] = 'Z'
        print(mar_arr)
        s = ""
        b = ""
        for k in range(n -1):
             s+= mar_arr[k]+'^'
        #print(s)
        for l in range(n -1):
            b+=')'
        final_eq = s+mar_arr[n-1]
        print(final_eq)
        break
    return eval(final_eq)

In [8]:
#Hamiltinian for N qubits

def Hamil(n):
    
    #Hamil = np.zeros((2^n, 2^n))
    #print("Hamil",Hamil)
    #Term_1 = np.zeros((2**n, 2**n))
    Term_1= 0
    #Term_1.shape
    #Term_2 = np.zeros((2**n, 2**n))
    Term_2 = 0
    for i   in  reversed(range(n)):
        for j in reversed(range(i)):
            
            #Term_1 = + nw*coff_zz(i,j,n)
            #print("term_1=",Term_1 )
            #print(Term_1.shape)
           #print(nw*coff_zz(i,j,n).shape)
            Term_1 += nw*coff_zz(i,j,n)
        Term_2 += nw*coff_z(i,n)
        #Term_2.shape
        #print("term_2=",Term_2 )
    Hamil = -(Term_1+Term_2)
        #Hamil += Hamil
    print("-------------------------------------")
    print("Hamil",Hamil)
    return Hamil
            

In [9]:
coff_z(5,6)

['I', 'I', 'I', 'I', 'I', 'Z']
I^I^I^I^I^Z


PauliOp(Pauli('IIIIIZ'), coeff=1.0)

In [10]:
#nw = random.uniform(0,1)
nw = 0.8421115487463554
print("Node waight is - ", nw)
print("                       ")
#ew = random.uniform(0,1)
ew = 0.6647028271173109
print("Endge waight is - ", ew)
H = Hamil(6)

Node waight is -  0.8421115487463554
                       
Endge waight is -  0.6647028271173109
I^I^I^I^Z^Z
I^I^I^Z^I^Z
I^I^Z^I^I^Z
I^Z^I^I^I^Z
Z^I^I^I^I^Z
['I', 'I', 'I', 'I', 'I', 'Z']
I^I^I^I^I^Z
I^I^I^Z^Z^I
I^I^Z^I^Z^I
I^Z^I^I^Z^I
Z^I^I^I^Z^I
['I', 'I', 'I', 'I', 'Z', 'I']
I^I^I^I^Z^I
I^I^Z^Z^I^I
I^Z^I^Z^I^I
Z^I^I^Z^I^I
['I', 'I', 'I', 'Z', 'I', 'I']
I^I^I^Z^I^I
I^Z^Z^I^I^I
Z^I^Z^I^I^I
['I', 'I', 'Z', 'I', 'I', 'I']
I^I^Z^I^I^I
Z^Z^I^I^I^I
['I', 'Z', 'I', 'I', 'I', 'I']
I^Z^I^I^I^I
['Z', 'I', 'I', 'I', 'I', 'I']
Z^I^I^I^I^I
-------------------------------------
Hamil -0.8421115487463554 * IIIIZZ
- 0.8421115487463554 * IIIZIZ
- 0.8421115487463554 * IIZIIZ
- 0.8421115487463554 * IZIIIZ
- 0.8421115487463554 * ZIIIIZ
- 0.8421115487463554 * IIIZZI
- 0.8421115487463554 * IIZIZI
- 0.8421115487463554 * IZIIZI
- 0.8421115487463554 * ZIIIZI
- 0.8421115487463554 * IIZZII
- 0.8421115487463554 * IZIZII
- 0.8421115487463554 * ZIIZII
- 0.8421115487463554 * IZZIII
- 0.8421115487463554 * ZIZIII


In [11]:
from qiskit.opflow.primitive_ops import PauliOp
#H_pauli= PauliOp.to_pauli_op(H_op)
#print(type(H_pauli))
print("---------------------------")
print(type(I))
print("---------------------------")
print(type(H))

---------------------------
<class 'qiskit.opflow.primitive_ops.pauli_op.PauliOp'>
---------------------------
<class 'qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp'>


In [12]:
import itertools
def generate_comb_pairs(n):
    comb=[]
    for i in range(n):
        comb += itertools.combinations(range(n),i+1)
    
    comb_list = [ list(t) for t in comb ]
    return comb_list

In [13]:
def build_encoding_circuit_rev(n,layer,param1):
    
    for i in reversed(range(n)):
        #print("i=",i)
    
        t_list = list(range(i+1))
       
        if i == 0:
            comb_pairs = generate_comb_pairs(i+1)
            #print(comb_pairs)
            t_list = list(range(i+1))
            #print(t_list)
            UG= UGate(-2*nw*param1[layer],0,0)
            qc.append(UG,t_list)
    #if i == 1:
        
        #for k in range(0,i):
           # qc.x(k)
        #qc.append(UG,t_list)
        #for k in range(0,i):
            #qc.x(k)
        #qc.barrier()
        if i == 1: 
            comb_pairs = generate_comb_pairs(i)
            #print(comb_pairs)
            #print("Creating firts gate , then anticontrols will follow.................")
            #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
            #UG= UGate(-2*nw*param1[layer],0,0).control(i)
            CUG = CU3Gate(-2*nw*param1[layer],0,0)
            qc.append(CUG,t_list)
            for sublist in comb_pairs:
                #print(sublist)
                for index in sublist:
                    #print(index)
                    qc.x(index)
            #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
            CUG = CU3Gate(-2*nw*param1[layer],0,0)
            qc.append(CUG,t_list)
            for sublist1 in comb_pairs:
                for index1 in sublist1:
                    qc.x(index1)
            qc.barrier()
        if i > 1: 
            comb_pairs = generate_comb_pairs(i)
            #print(comb_pairs)
        #t_list = comb_pairs[-1]
            #print("Creating firts gate , then anticontrols will follow.................")
            #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
            UG= UGate(-2*nw*param1[layer],0,0).control(i)
            qc.append(UG,t_list)
            for sublist in comb_pairs:
                #print(sublist)
                for index in sublist:
                    #print("Adding Xgate at", index )
                    qc.x(index)
               #print("Adding u Gate")
                #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
                UG= UGate(-2*nw*param1[layer],0,0).control(i)
                qc.append(UG,t_list)
                for index1 in sublist:
                    #print("Adding Xgate at", index1)
                    qc.x(index1)
                qc.barrier()    
        qc.barrier()
        qc.barrier()
    
        

In [14]:
def build_encoding_circuit(n,layer,param1):
    
    for i in range(n):
        #print("i=",i)
    
        t_list = list(range(i+1))
       
        if i == 0:
            comb_pairs = generate_comb_pairs(i+1)
            #print(comb_pairs)
            t_list = list(range(i+1))
            #print(t_list)
            UG= UGate(-2*nw*param1[layer],0,0)
            qc.append(UG,t_list)
    #if i == 1:
        
        #for k in range(0,i):
           # qc.x(k)
        #qc.append(UG,t_list)
        #for k in range(0,i):
            #qc.x(k)
        #qc.barrier()
        if i == 1: 
            comb_pairs = generate_comb_pairs(i)
            #print(comb_pairs)
        #t_list = comb_pairs[-1]
            #print("Creating firts gate , then anticontrols will follow.................")
            #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
            CUG = CU3Gate(-2*nw*param1[layer],0,0)
            qc.append(CUG,t_list)
            for sublist in comb_pairs:
                #print(sublist)
                for index in sublist:
                    #print(index)
                    qc.x(index)
            #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
            CUG = CU3Gate(-2*nw*param1[layer],0,0)
            qc.append(CUG,t_list)
            for sublist1 in comb_pairs:
                for index1 in sublist1:
                    qc.x(index1)
            qc.barrier()
        if i > 1: 
            comb_pairs = generate_comb_pairs(i)
            #print(comb_pairs)
        #t_list = comb_pairs[-1]
            #print("Creating firts gate , then anticontrols will follow.................")
            #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
            UG= UGate(-2*nw*param1[layer],0,0).control(i)
            qc.append(UG,t_list)
            for sublist in comb_pairs:
                #print(sublist)
                for index in sublist:
                    #print("Adding Xgate at", index )
                    qc.x(index)
                #print("Adding u Gate")
                #UG= UGate(0,math.pi/2,-math.pi/2).control(i)
                UG= UGate(-2*nw*param1[layer],0,0).control(i)
                qc.append(UG,t_list)
                for index1 in sublist:
                    #print("Adding Xgate at", index1)
                    qc.x(index1)
                qc.barrier()    
        qc.barrier()
        qc.barrier()

In [15]:
def u3_ansatz2(qc,n,layers,param1=[]):
# Parameters    qc     : quantum circuit   
#               n      : number of qubits    
# layers               : number of layers of ansatz    
# Returns       qc     : modified quantum circuit    '''
 
    if  len(param1)== 0 or len(param2)==0 or len(param3)==0 :
            param1=ParameterVector('x',length=(layers)*n*3)#u3 has 3 parameters so layers*qubits*u3params
            #param1=ParameterVector('x',length=(layers))
            #param2=ParameterVector('y',length=(layers))
            #param3=ParameterVector('a',length=(layers))
           
    for layer in range(layers):
        
        build_encoding_circuit(n,layer,param1)

        for i   in  reversed(range(n)):
            for j in reversed(range(i)):
                #print("i=",i)
                #print("j=",j)
                qc.cx(i,j)
                qc.u(-2*nw*param1[layer],0,0,j)
                #qc.u(0,-2*ZZ_65*param1[layer],0,j)
                qc.cx(i,j)
                qc.barrier()
        for i   in  reversed(range(n)): 
            qc.u(-2*ew*param1[layer],0,0,i)
        qc.barrier()

        build_encoding_circuit_rev(n,layer,param1)
    return qc

In [16]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer_type = 'COBYLA'

if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=1000)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=500)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=500)

In [17]:
locs = 3
n = locs*(locs -1)
qc = BuildCircuit(locs)
qc = u3_ansatz2(qc,6,1)

----------Staring the Process and Allocating Qubits -------------
                                                                 
Total Qubits allocated is :    6


In [18]:
intermediate_info = {"nfev": [], "parameters": [], "energy": [], "stddev": []}


def callback(nfev, parameters, energy, stddev):
    intermediate_info["nfev"].append(nfev)
    intermediate_info["parameters"].append(parameters)
    intermediate_info["energy"].append(energy)
    intermediate_info["stddev"].append(stddev)

In [19]:
#nw = random.uniform(0,1)
nw = 0.8421115487463554
print("Node waight is - ", nw)
print("                       ")
#ew = random.uniform(0,1)
ew = 0.6647028271173109
print("Endge waight is - ", ew)

Node waight is -  0.8421115487463554
                       
Endge waight is -  0.6647028271173109


In [20]:
from qiskit.circuit.library import UGate
from qiskit.algorithms import VQE
from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit import Aer
from qiskit.opflow import X, Z, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.providers.aer import QasmSimulator
#backend = Aer.get_backend('qasm_simulator')
#backend = Aer.get_backend('qasm_simulator')

#Details

#VQE.initial_point = None
#vqe = VQE(H_op, qc, optimizer)
#vqe_result = vqe.run(backend)



#np.random.seed(10)  # seed for reproducibility
#initial_point = np.random.random(qc.num_parameters)

#backend = Aer.get_backend('statevector_simulator')
#qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

seed = np.random.seed(10)
algorithm_globals.random_seed = seed
#qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
qi =QuantumInstance(QasmSimulator(), shots=1,seed_simulator=seed)

slsqp = SLSQP(maxiter=1000)
vqe = VQE(qc, optimizer=optimizer, quantum_instance=qi)

#local_vqe = VQE(
#    ansatz= qc,
#    optimizer=optimizer,
#    initial_point= None,
#    quantum_instance=qi,
#    callback=callback,
#)

In [21]:
# qc.draw(output='mpl')

In [22]:
qc.num_qubits

6

In [23]:
# H.num_qubits

In [24]:
local_result = vqe.compute_minimum_eigenvalue(H)
print(local_result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 12,
    'eigenstate': {'000100': 1.0},
    'eigenvalue': (-17.684342523673468+0j),
    'optimal_parameters': {ParameterVectorElement(x[0]): 0.060756422948185396},
    'optimal_point': array([0.06075642]),
    'optimal_value': -17.684342523673468,
    'optimizer_evals': None,
    'optimizer_time': 7.287761211395264}


In [25]:
from qiskit.algorithms import NumPyMinimumEigensolver
mes = NumPyMinimumEigensolver()
mes.compute_minimum_eigenvalue(H)

In [26]:
print(mes.compute_minimum_eigenvalue(H))

{   'aux_operator_eigenvalues': None,
    'eigenstate': VectorStateFn(Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False),
    'eigenvalue': (-17.684342523673468+0j)}
